In [0]:
from tensorflow import keras
import numpy as np
import os
import imutils

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, Reshape, Flatten
from tensorflow.keras.layers import InputLayer, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

In [0]:
#to delete the git files
!rm -rf kerasProjectPedestrian

In [3]:
#to clone the repo with all the pictures and annotations
!git clone https://github.com/BarHanSolo/kerasProjectPedestrian.git

Cloning into 'kerasProjectPedestrian'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 4047 (delta 2), reused 0 (delta 0), pack-reused 4040
Receiving objects: 100% (4047/4047), 6.28 GiB | 35.92 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Checking out files: 100% (3982/3982), done.


In [0]:
# --- Get list of all files of desired format(xml, mp4 or whatever) in the folder ---

def getListOf(format):
  path = "kerasProjectPedestrian"
  files = []
  for r, d, f in os.walk(path):
    for file in f:
        if format in file:
            files.append(os.path.join(r, file))
  return files

In [0]:
# --- Get list of images ---

import os

# defining global variable path
# Location of my dataset
imagePath = "kerasProjectPedestrian"

def loadImages(path, video):
    '''Put files into lists and return them as one list with all images 
     in the folder'''
    imageFiles = sorted([os.path.join(path, video, file)
                          for file in os.listdir(path + "/" + video)
                          if file.endswith('.png')])
    return imageFiles


In [7]:
listOfVideos = sorted(getListOf("xml"))
print(listOfVideos)
listOfVideosImages = []
for video in listOfVideos:
  videoShort = video[23:33]
  images = loadImages(imagePath, videoShort)
  listOfVideosImages.append(images)
print(listOfVideosImages)
print(len(listOfVideosImages[0]))
print(len(listOfVideosImages[1]))
print(len(listOfVideosImages[2]))

['kerasProjectPedestrian/video_0001.xml', 'kerasProjectPedestrian/video_0002.xml', 'kerasProjectPedestrian/video_0003.xml', 'kerasProjectPedestrian/video_0004.xml', 'kerasProjectPedestrian/video_0005.xml', 'kerasProjectPedestrian/video_0006.xml', 'kerasProjectPedestrian/video_0007.xml', 'kerasProjectPedestrian/video_0008.xml', 'kerasProjectPedestrian/video_0009.xml', 'kerasProjectPedestrian/video_0010.xml', 'kerasProjectPedestrian/video_0011.xml', 'kerasProjectPedestrian/video_0012.xml', 'kerasProjectPedestrian/video_0013.xml', 'kerasProjectPedestrian/video_0014.xml', 'kerasProjectPedestrian/video_0015.xml', 'kerasProjectPedestrian/video_0016.xml', 'kerasProjectPedestrian/video_0017.xml']
[['kerasProjectPedestrian/video_0001/00000.png', 'kerasProjectPedestrian/video_0001/00001.png', 'kerasProjectPedestrian/video_0001/00002.png', 'kerasProjectPedestrian/video_0001/00003.png', 'kerasProjectPedestrian/video_0001/00004.png', 'kerasProjectPedestrian/video_0001/00005.png', 'kerasProjectPedes

In [8]:
# --- Generate objects with attributes for each frame --- 

import xml.etree.ElementTree as ET
objectsArray = [] 
Y = []
objects = []
annotationList = getListOf("xml")
videoNumber = 0
for video in listOfVideos:
  videoLength = [None]*len(listOfVideosImages[videoNumber])
  annotationFilePath = sorted(annotationList)[videoNumber]
  videoNumber+=1
  tree = ET.parse(annotationFilePath)
  root = tree.getroot()

  #for node in tree.iter('attribute'):
  for node in root.iter('box'):
    nodeAttributes = {}
    y = None
    frameNumber = node.attrib['frame']

    for node in tree.iter('attribute'):
      for attributeName in node.attrib:
        if attributeName == 'name' and node.attrib["name"] != 'cross':

          nodeAttributes[node.attrib["name"]] = node.text

        elif attributeName == 'name' and node.attrib["name"] == 'cross':
          y = node.text
      
    objectsArray.append(nodeAttributes)
    Y.append(y)
    try:
      videoLength[int(frameNumber)]=y
    except:
      print(len(videoLength))
      print("kurdebele" + frameNumber)
      print(videoNumber)
    
    #print('Y', len(Y), Y[0])
    #print('objectsArray!!!', objectsArray)
    #print('')
  objects.append(videoLength)
print(len(objects))
print(len(objects[0]))

17
600


In [9]:
print(len(Y))
print(len(objects))
print(len(objects[2]))

12297
17
210


In [10]:
# --- Connect frames to their attributes ---

dataSet = []

for i, videoImages in enumerate(listOfVideosImages):
  for j, videoImage in enumerate(videoImages):
    frameObject = objectsArray[j]
    dataSet.append(frameObject)
    
    #for image in videoImages:
    frameObject['image'] = videoImage
  
print(dataSet)
print(len(dataSet))

[{'id': '0_1_4', 'old_id': 'ped1', 'look': 'not-looking', 'reaction': '__undefined__', 'action': 'walking', 'hand_gesture': '__undefined__', 'occlusion': 'none', 'nod': '__undefined__', 'image': 'kerasProjectPedestrian/video_0017/00000.png'}, {'id': '0_1_4', 'old_id': 'ped1', 'look': 'not-looking', 'reaction': '__undefined__', 'action': 'walking', 'hand_gesture': '__undefined__', 'occlusion': 'none', 'nod': '__undefined__', 'image': 'kerasProjectPedestrian/video_0017/00001.png'}, {'id': '0_1_4', 'old_id': 'ped1', 'look': 'not-looking', 'reaction': '__undefined__', 'action': 'walking', 'hand_gesture': '__undefined__', 'occlusion': 'none', 'nod': '__undefined__', 'image': 'kerasProjectPedestrian/video_0017/00002.png'}, {'id': '0_1_4', 'old_id': 'ped1', 'look': 'not-looking', 'reaction': '__undefined__', 'action': 'walking', 'hand_gesture': '__undefined__', 'occlusion': 'none', 'nod': '__undefined__', 'image': 'kerasProjectPedestrian/video_0017/00003.png'}, {'id': '0_1_4', 'old_id': 'ped1

In [0]:
# --- Sample code for resizing the input images --- 

# resize the frame to have a maximum width of 416 pixels (the
# less data we have, the faster we can process it), then convert
# the frame from BGR to RGB for dlib
#frame = imutils.resize(frame, width=416)
#rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

In [0]:
# --- Returns array without dictionary ---

def dedict(listOfDictionaries):
  X = []
  for element in listOfDictionaries:
    X2 = []
    for key, value in element.items():
      temp = [key,value]
      X2.append(temp)
    X.append(X2)
  return X

In [0]:
def crossingConverter(lists):
  newNewList = []
  for list in lists:
    newList = []
    for item in list:
      if item =='not-crossing':
        newList.append(0)
      else:
        newList.append(1)
    newList = np.array(newList)
    newNewList.append(newList)
  return newNewList

In [0]:
def categorizeOneHot(new_array):
  instances = np.array(new_array)
  nsamples, nx, ny = instances.shape
  d2_train_dataset = instances.reshape((nsamples,nx*ny))
  enc = OneHotEncoder(handle_unknown='ignore', sparse = False)
  new = enc.fit_transform(d2_train_dataset)
  return new

In [0]:
def convertX(X):
  new_array = dedict(X)
  new_array2 = categorizeOneHot(new_array)
  new_array2 = np.array(new_array2)
  return new_array2

def convertY(objects):
  objects2 = crossingConverter(objects)
  objects2 = np.array(objects2)
  flat_list = []
  for sublist in objects2:
      for item in sublist:
          flat_list.append(item)
  flat_list = np.array(flat_list)
  return flat_list

In [0]:
#definition of the model itself
def create_model(hidden_size, input_shape):
  model = Sequential()
  model.add(InputLayer(input_shape=input_shape, batch_size=None, dtype=None, input_tensor=None))
  #model.add(Conv2D(2, 3, activation='relu', input_shape=input_shape))
  model.add(Dense(8, activation='softmax'))
  model.add(Dense(hidden_size, activation='relu'))
  model.add(Dense(hidden_size, activation='relu'))
  return model

In [17]:
# --- Create model ---

# X contains features of images
# Y contains crossing feature

X = dataSet.copy()
Y = objects.copy()

#those functions hide all operations that are needed on the datasets to apply them to the model
X = convertX(X)
Y = convertY(Y)

#some variables befor the model
hidden_size = 32
input_shape = X.shape
loss = 'binary_crossentropy'
optimizer = 'adam'
metrics = ['accuracy','binary_accuracy']

model = create_model(hidden_size, input_shape)
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

model.fit(X, Y, batch_size=32, epochs=100)

model.summary()



Epoch 1/100
124/124 [==============================] - 0s 2ms/step - loss: 3.2218 - accuracy: 0.2227 - binary_accuracy: 0.6652
Epoch 2/100
124/124 [==============================] - 0s 2ms/step - loss: 2.7605 - accuracy: 0.1750 - binary_accuracy: 0.6652
Epoch 3/100
124/124 [==============================] - 0s 2ms/step - loss: 2.6191 - accuracy: 0.0081 - binary_accuracy: 0.6652
Epoch 4/100
124/124 [==============================] - 0s 2ms/step - loss: 2.6182 - accuracy: 0.0328 - binary_accuracy: 0.6652
Epoch 5/100
124/124 [==============================] - 0s 2ms/step - loss: 2.6182 - accuracy: 0.0510 - binary_accuracy: 0.6652
Epoch 6/100
124/124 [==============================] - 0s 2ms/step - loss: 2.6178 - accuracy: 0.0376 - binary_accuracy: 0.6652
Epoch 7/100
124/124 [==============================] - 0s 2ms/step - loss: 2.6177 - accuracy: 0.0361 - binary_accuracy: 0.6652
Epoch 8/100
124/124 [==============================] - 0s 2ms/step - loss: 2.6175 - accuracy: 0.0338 - binary_a